In [1]:
import pandas as pd
import numpy as np

In [2]:
N = 100000000

In [3]:
HDF_KWARGS = {
    'data_columns': True,  # 便于列查询
    'append': True,  # 数据可添加
    'complevel': 9,
    'complib': 'blosc:blosclz', # 默认库。速度最快
}

In [4]:
codes = [str(i).zfill(6) for i in range(1,1000)]

In [5]:
df = pd.DataFrame({
    '日期':
    pd.date_range('1990-11-01 00:00', periods=N, freq='s'),
    '股票代码':
    np.random.choice(codes, N),
    '涨跌幅':
    np.random.random(N),
    '股价':
    np.random.random(N),
})

In [6]:
df.set_index(['日期', '股票代码'], inplace=True)

In [7]:
df.tail()

,,涨跌幅,股价
日期,股票代码,,
1994-01-01 09:46:35,000085,0.268336,0.046985
1994-01-01 09:46:36,000862,0.086593,0.794695
1994-01-01 09:46:37,000097,0.726068,0.739594
1994-01-01 09:46:38,000179,0.166853,0.956411
1994-01-01 09:46:39,000117,0.251955,0.883251


# 写入

In [8]:
# 带索引写入 7min
# 不带索引 50s
store = pd.HDFStore('large_cn.h5')
store.append("df", df, index=False, **HDF_KWARGS)

C:\Users\liude\anaconda3\envs\zipline\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '股票代码'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
C:\Users\liude\anaconda3\envs\zipline\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '日期'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
C:\Users\liude\anaconda3\envs\zipline\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '涨跌幅'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will stil

## 创建索引

In [9]:
# 创建索引 5min35s
store.create_table_index('df', optlevel=9, kind='full')

# 查询

## 主坐标

In [10]:
# 15ms
store.select("df", "日期>=pd.Timestamp('1994-01-01 09:46:35') and 日期<=pd.Timestamp('1994-01-01 12:46:35')")

,,涨跌幅,股价
日期,股票代码,,
1994-01-01 09:46:35,000085,0.268336,0.046985
1994-01-01 09:46:36,000862,0.086593,0.794695
1994-01-01 09:46:37,000097,0.726068,0.739594
1994-01-01 09:46:38,000179,0.166853,0.956411
1994-01-01 09:46:39,000117,0.251955,0.883251


## 次坐标

In [11]:
# 7.5s 20w rows
store.select("df","股票代码 = ['000001','000099']")

,,涨跌幅,股价
日期,股票代码,,
1990-11-01 00:15:08,000099,0.213317,0.843223
1990-11-01 00:26:38,000099,0.194780,0.474543
1990-11-01 00:32:57,000001,0.728373,0.500865
1990-11-01 00:33:20,000001,0.656082,0.012898
1990-11-01 00:36:02,000001,0.651777,0.446269
...,...,...,...
1994-01-01 09:02:08,000099,0.603316,0.533297
1994-01-01 09:06:20,000001,0.658950,0.163983
1994-01-01 09:21:13,000099,0.661527,0.024193


In [14]:
store.select("df","股票代码 = '000099'")

,,涨跌幅,股价
日期,股票代码,,
1990-11-01 00:15:08,000099,0.213317,0.843223
1990-11-01 00:26:38,000099,0.194780,0.474543
1990-11-01 00:53:35,000099,0.212584,0.091240
1990-11-01 01:39:26,000099,0.341710,0.543048
1990-11-01 02:17:49,000099,0.071071,0.415948
...,...,...,...
1994-01-01 08:38:02,000099,0.043753,0.922558
1994-01-01 09:02:08,000099,0.603316,0.533297
1994-01-01 09:21:13,000099,0.661527,0.024193


In [12]:
# 44s 2000w rows
store.select("df","股价 > 0.8")

,,涨跌幅,股价
日期,股票代码,,
1990-11-01 00:00:00,000749,0.265795,0.871326
1990-11-01 00:00:01,000555,0.860194,0.991408
1990-11-01 00:00:02,000314,0.875841,0.864301
1990-11-01 00:00:03,000063,0.760080,0.928494
1990-11-01 00:00:06,000561,0.571876,0.904741
...,...,...,...
1994-01-01 09:46:21,000346,0.396784,0.960244
1994-01-01 09:46:23,000549,0.551476,0.988917
1994-01-01 09:46:29,000591,0.279189,0.930674


## 联合

In [13]:
# 15s 630w rows
store.select("df", "日期 >= pd.Timestamp('1993-01-01 09:46:35') and 股价 > 0.8")

,,涨跌幅,股价
日期,股票代码,,
1993-01-01 09:46:40,000335,0.605382,0.899732
1993-01-01 09:46:58,000269,0.205313,0.942192
1993-01-01 09:47:00,000781,0.663895,0.816324
1993-01-01 09:47:03,000168,0.786497,0.862379
1993-01-01 09:47:07,000967,0.797368,0.965377
...,...,...,...
1994-01-01 09:46:21,000346,0.396784,0.960244
1994-01-01 09:46:23,000549,0.551476,0.988917
1994-01-01 09:46:29,000591,0.279189,0.930674
